**Процедура стандартизации данных**

In [1]:
import pandas as pd

# Определение

Пусть имеется множество измерений определенного признака $x_i, i=\overline{1,n}$. Тогда стандартизацией такого ряда называется преобразование по формуле:

$$\tilde{x}_{i} = \frac{x_{i} - \bar{x}}{\sigma}. \tag{1}$$

Где:
- $\bar{x}$ - среднее арифметческое рассматртваемого ряда;
- $\sigma$ - стандартное отклонение.

Получется, что выражение $(1)$ может быть переписано следующим образом:

$$\tilde{x}_{i} = \frac{x_{i} - \bar{x}}{\sqrt{\frac{1}{n} (x_i - \bar{x})^2}}. \tag{2}$$

Иногда предпочитают не отнимать среднее арифметическое в числителе, тогда формула $(1)$ принимает вид:

$$\tilde{x}_i  = \frac{x_i}{\sigma}. \tag{3}$$

# Свойсва результата

Величина получаемая в результате применения формулы $(1)$ получает следующие свойсва:

1. **Среднее артфииетисеское результата равняется 0**

вафыоавдыа

2. **аыавыа**